In [1]:
from statsmodels.tsa.stattools import grangercausalitytests
import csv
import requests
import pandas as pd
import numpy as np
import os

In [2]:
gtrends_data = pd.read_csv("gtrends_data/gtrends_weekly_01-01-12_10-29-18.csv", names=["ds", "y"])
gtrends_data['y'] = gtrends_data['y'].replace({"<1": "0"})

In [3]:
POLONIEX_OHLCV_BASEURL = 'https://poloniex.com/public?command=returnChartData&currencyPair='

"""
https://poloniex.com/public?command=returnChartData&currencyPair=BTC_POT&start=1435699200&end=9999999999&period=14400

"""

def get_ohlcv_poloniex(pair='USDT_ETH', start=1435699200, end=9999999999, period=900):
    """
    returns ohlcv data for poloniex as pandas dataframe
    convert to unix timestamp using https://coderstoolbox.net/unixtimestamp/
    :param pair: str pair on poloniex
    :param start: int unix timestamp of beginning time
    :param end: int unix timestamp of ending time
    :param period: int candle width in seconds
    :return: pandas df of ohlcv data from poloniex for specified pair, times, and period
    """
    query = POLONIEX_OHLCV_BASEURL + pair + '&start=' + str(start) + '&end=' + str(end) + '&period=' + str(period)
    resp = requests.get(query)

    if resp.status_code != 200:
        raise requests.ApiError('GET /tasks/ {}'.format(resp.status_code))

    return pd.DataFrame(resp.json())

In [4]:
df = get_ohlcv_poloniex(pair='USDT_ETH', start=0, end=9999999999, period=900)
df = df[['date','close','volume']]
df['ds'] = pd.to_datetime(df['date'], unit='s')
df['log'] = np.log(df['close'])
df['vol'] = df['log'].rolling(5).std()
df.dropna();

In [5]:
path = 'gtrends_data/'

In [6]:
series = 'close'
for i in  [1,2,4,6,8]:
    for isAbs in [True, False]:
        data = df[series]
        if isAbs:
            data.apply(lambda x:abs(x))
        combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
        combined = combined.dropna(how='any')
        print (series)
        print (i)
        print (grangercausalitytests(combined, i))

close
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0065  , p=0.9358  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.0068  , p=0.9344  , df=1
likelihood ratio test: chi2=0.0068  , p=0.9344  , df=1
parameter F test:         F=0.0065  , p=0.9358  , df_denom=78, df_num=1
{1: ({'ssr_ftest': (0.006525453221397484, 0.9358236061867066, 78.0, 1), 'ssr_chi2test': (0.006776432191451233, 0.9343929761834533, 1), 'lrtest': (0.006776148750304856, 0.9343943451899271, 1), 'params_ftest': (0.006525453221390623, 0.9358236061867915, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x11aa27780>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x11aa27eb8>, array([[0., 1., 0.]])])}
close
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.0065  , p=0.9358  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.0068  , p=0.9344  , df=1
likelihood ratio test: chi2=0.0068  , p=0.9344  ,

likelihood ratio test: chi2=0.0261  , p=1.0000  , df=6
parameter F test:         F=0.0036  , p=1.0000  , df_denom=63, df_num=6

Granger Causality
number of lags (no zero) 7
ssr based F test:         F=0.0149  , p=1.0000  , df_denom=60, df_num=7
ssr based chi2 test:   chi2=0.1300  , p=1.0000  , df=7
likelihood ratio test: chi2=0.1299  , p=1.0000  , df=7
parameter F test:         F=0.0149  , p=1.0000  , df_denom=60, df_num=7

Granger Causality
number of lags (no zero) 8
ssr based F test:         F=0.1467  , p=0.9964  , df_denom=57, df_num=8
ssr based chi2 test:   chi2=1.5233  , p=0.9923  , df=8
likelihood ratio test: chi2=1.5078  , p=0.9926  , df=8
parameter F test:         F=0.1467  , p=0.9964  , df_denom=57, df_num=8
{1: ({'ssr_ftest': (0.006525453221397484, 0.9358236061867066, 78.0, 1), 'ssr_chi2test': (0.006776432191451233, 0.9343929761834533, 1), 'lrtest': (0.006776148750304856, 0.9343943451899271, 1), 'params_ftest': (0.006525453221390623, 0.9358236061867915, 78.0, 1.0)}, [<statsmo

In [7]:
series = 'volume'
for i in  [1,2,4,6,8]:
    for isAbs in [True, False]:
        data = df[series]
        if isAbs:
            data.apply(lambda x:abs(x))
        combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
        combined = combined.dropna(how='any')
        print (series)
        print (i)
        print (grangercausalitytests(combined, i))

volume
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1879  , p=0.6659  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.1951  , p=0.6587  , df=1
likelihood ratio test: chi2=0.1948  , p=0.6589  , df=1
parameter F test:         F=0.1879  , p=0.6659  , df_denom=78, df_num=1
{1: ({'ssr_ftest': (0.18785116775279065, 0.6659074190116527, 78.0, 1), 'ssr_chi2test': (0.1950762126663595, 0.6587248887845409, 1), 'lrtest': (0.19484168340989072, 0.6589171097396751, 1), 'params_ftest': (0.1878511677527989, 0.6659074190116527, 78.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1229abdd8>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1229ab400>, array([[0., 1., 0.]])])}
volume
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1879  , p=0.6659  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.1951  , p=0.6587  , df=1
likelihood ratio test: chi2=0.1948  , p=0.6589  , df=

volume
6

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1879  , p=0.6659  , df_denom=78, df_num=1
ssr based chi2 test:   chi2=0.1951  , p=0.6587  , df=1
likelihood ratio test: chi2=0.1948  , p=0.6589  , df=1
parameter F test:         F=0.1879  , p=0.6659  , df_denom=78, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1512  , p=0.8599  , df_denom=75, df_num=2
ssr based chi2 test:   chi2=0.3226  , p=0.8510  , df=2
likelihood ratio test: chi2=0.3220  , p=0.8513  , df=2
parameter F test:         F=0.1512  , p=0.8599  , df_denom=75, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.0797  , p=0.9708  , df_denom=72, df_num=3
ssr based chi2 test:   chi2=0.2625  , p=0.9669  , df=3
likelihood ratio test: chi2=0.2621  , p=0.9670  , df=3
parameter F test:         F=0.0797  , p=0.9708  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.0685  , p=0.991

likelihood ratio test: chi2=0.2621  , p=0.9670  , df=3
parameter F test:         F=0.0797  , p=0.9708  , df_denom=72, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.0685  , p=0.9912  , df_denom=69, df_num=4
ssr based chi2 test:   chi2=0.3097  , p=0.9892  , df=4
likelihood ratio test: chi2=0.3091  , p=0.9892  , df=4
parameter F test:         F=0.0685  , p=0.9912  , df_denom=69, df_num=4

Granger Causality
number of lags (no zero) 5
ssr based F test:         F=0.0613  , p=0.9974  , df_denom=66, df_num=5
ssr based chi2 test:   chi2=0.3573  , p=0.9964  , df=5
likelihood ratio test: chi2=0.3565  , p=0.9964  , df=5
parameter F test:         F=0.0613  , p=0.9974  , df_denom=66, df_num=5

Granger Causality
number of lags (no zero) 6
ssr based F test:         F=0.0627  , p=0.9990  , df_denom=63, df_num=6
ssr based chi2 test:   chi2=0.4535  , p=0.9984  , df=6
likelihood ratio test: chi2=0.4522  , p=0.9984  , df=6
parameter F test:         F=0.0627  , p=0.999

        0.]])])}


In [8]:
series = 'vol'
for i in  [1,2,4,6,8]:
    for isAbs in [True, False]:
        data = df[series]
        if isAbs:
            data.apply(lambda x:abs(x))
        combined = pd.concat([data,gtrends_data['y'][2:]], axis=1)
        combined = combined.dropna(how='any')
        print (series)
        print (i)
        print (grangercausalitytests(combined, i))

vol
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.1525  , p=0.6962  , df=1
likelihood ratio test: chi2=0.1523  , p=0.6963  , df=1
parameter F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1
{1: ({'ssr_ftest': (0.14667984431396486, 0.7027977481270142, 76.0, 1), 'ssr_chi2test': (0.1524698381684635, 0.6961861783215509, 1), 'lrtest': (0.15232289396828946, 0.6963253281813486, 1), 'params_ftest': (0.14667984431389797, 0.7027977481270796, 76.0, 1.0)}, [<statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1229c9e48>, <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x1229c93c8>, array([[0., 1., 0.]])])}
vol
1

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.1525  , p=0.6962  , df=1
likelihood ratio test: chi2=0.1523  , p=0.6963  , df=1
par

vol
6

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.1525  , p=0.6962  , df=1
likelihood ratio test: chi2=0.1523  , p=0.6963  , df=1
parameter F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1781  , p=0.8372  , df_denom=73, df_num=2
ssr based chi2 test:   chi2=0.3805  , p=0.8267  , df=2
likelihood ratio test: chi2=0.3796  , p=0.8271  , df=2
parameter F test:         F=0.1781  , p=0.8372  , df_denom=73, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.1643  , p=0.9201  , df_denom=70, df_num=3
ssr based chi2 test:   chi2=0.5423  , p=0.9095  , df=3
likelihood ratio test: chi2=0.5404  , p=0.9099  , df=3
parameter F test:         F=0.1643  , p=0.9201  , df_denom=70, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.1312  , p=0.9705  

vol
8

Granger Causality
number of lags (no zero) 1
ssr based F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1
ssr based chi2 test:   chi2=0.1525  , p=0.6962  , df=1
likelihood ratio test: chi2=0.1523  , p=0.6963  , df=1
parameter F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1781  , p=0.8372  , df_denom=73, df_num=2
ssr based chi2 test:   chi2=0.3805  , p=0.8267  , df=2
likelihood ratio test: chi2=0.3796  , p=0.8271  , df=2
parameter F test:         F=0.1781  , p=0.8372  , df_denom=73, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.1643  , p=0.9201  , df_denom=70, df_num=3
ssr based chi2 test:   chi2=0.5423  , p=0.9095  , df=3
likelihood ratio test: chi2=0.5404  , p=0.9099  , df=3
parameter F test:         F=0.1643  , p=0.9201  , df_denom=70, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.1312  , p=0.9705  

likelihood ratio test: chi2=0.1523  , p=0.6963  , df=1
parameter F test:         F=0.1467  , p=0.7028  , df_denom=76, df_num=1

Granger Causality
number of lags (no zero) 2
ssr based F test:         F=0.1781  , p=0.8372  , df_denom=73, df_num=2
ssr based chi2 test:   chi2=0.3805  , p=0.8267  , df=2
likelihood ratio test: chi2=0.3796  , p=0.8271  , df=2
parameter F test:         F=0.1781  , p=0.8372  , df_denom=73, df_num=2

Granger Causality
number of lags (no zero) 3
ssr based F test:         F=0.1643  , p=0.9201  , df_denom=70, df_num=3
ssr based chi2 test:   chi2=0.5423  , p=0.9095  , df=3
likelihood ratio test: chi2=0.5404  , p=0.9099  , df=3
parameter F test:         F=0.1643  , p=0.9201  , df_denom=70, df_num=3

Granger Causality
number of lags (no zero) 4
ssr based F test:         F=0.1312  , p=0.9705  , df_denom=67, df_num=4
ssr based chi2 test:   chi2=0.5953  , p=0.9636  , df=4
likelihood ratio test: chi2=0.5929  , p=0.9638  , df=4
parameter F test:         F=0.1312  , p=0.970